<a href="https://colab.research.google.com/github/pevu97/Housing-Price-Regression-NN/blob/main/notebooks/Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

!git clone https://github.com/pevu97/Housing-Price-Regression-NN.git


np.set_printoptions(precision=3, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
tf.__version__

Cloning into 'Housing-Price-Regression-NN'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 52 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 417.43 KiB | 2.84 MiB/s, done.
Resolving deltas: 100% (8/8), done.


'2.18.0'

In [13]:
raw_datasets = pd.read_csv('Housing-Price-Regression-NN/data/housing.csv')